In [3]:
!pip install stanza
import stanza
import sqlite3

In [4]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL") #we take into account "NULL" titles, to keep the same structure as original DB.
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [26]:
#Delimitation of titles (later recognizing them)
import re
'''def find_delimit_subtit(text):
  #subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r' | ', text)
  subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r'" | ', text)## N: He restituído la comilla de cierre que te habías perdido, igual es problemático para los que no ponían comillas... lo vamos viendo
  return subtitle_'''

def subtitle_format(matchobj):
  match = matchobj.group(0)
  if re.fullmatch(r'\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*-\s*"\s*', match):
    return ' | "'
  elif re.fullmatch(r'\s*-\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*\[\.\.\.\]\s*"\s*-\s*', match):
    submatch = re.fullmatch(r'(\s*)\[\.\.\.\]\s*"\s*-\s*', match)
    return submatch.group(1)+'[...]" | '
    
def formatting_subtitles(text):
    subtit_re = r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*'
    new_text = re.sub(subtit_re, subtitle_format, text)
    return new_text
    
    
encontrar_subtit = [formatting_subtitles(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
def subadhoc(matchobj):
  return ' "'+ matchobj.group(0)[1:]     
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('"Alzad', '| "Alzad')
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')
    #encontrar_subtit[i] = re.sub(r'[^|]+\]"', subadhoc, encontrar_subtit[i])##N: añadido mío (con la definición de subadhoc)

print(encontrar_subtit[249])
print(encontrar_subtit[207])


Inauguración del teatro de D. José Piquer-Oda | "Alzad vuestras tumbas [...]"
Matrimonios por anuncios | Los baños de Biarritz | "Hace pocos días [...]"
Al Sr. D. R. P. Enviándole mis obras
La mujer piadosa (Fragmentos)
Desvarío
Soneto. "¡Gloria y pobreza! El inmortal Quintana..."
Balada. La despedida de una hija
Un viaje a la Luna. Prólogo: La pesadilla de Mocquet
En el álbum de la señora M…
La primavera
La perejilera
Un viaje a la Luna. (Continuación)
Al señor Don B. M.
A orillas del Manzanares. La súplica
Un viaje a la Luna. (Continuación)
Fragmentos de la Rosa de Alejandra
Gente morena
Un viaje a la Luna. (Continuación)
Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo
A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños
Costumbres del siglo XIX. I
Margarita. Novela bretona
Los escondidos y la tapada. Fragmentos
A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum
Costumbres del siglo XIX.II
Margarita. Novela bretona. (Con

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_431641/3979252347.py:3: SyntaxWarning: invalid escape sequence '\s'
  '''def find_delimit_subtit(text):


In [ ]:
#we continue preparing each title to process it through model (tok,lemm,pos,depparse,ner)
import re

def clean_per_title(title):
  cleaned_text = re.sub(
    r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*',
    ' ',
    title)
  cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  return cleaned_text.strip()

cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

#print(cleaned_rows)

for row in cleaned_rows:
  if " | " in row:
    cleaned_rows[i] = row.split(" | ")

#for row in cleaned_rows:
  #print(row, "\n")

In [ ]:
#we start building up the function to process each title and each subtitle
# START HERE TOMORROW, NOT ENDED 18.03.2025.
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [processing_1(row, nlp_es) for row in titles_list]

processed_titles_es